# Using APIs to access LLMs

Assignment 3 - Prompt Engineering for In-Context Learning

In [ ]:
# pip install openai
!pip3 install openai

#!pip3 install --upgrade pip

In [ ]:
"""Run this model in Python

> pip install openai
"""
import os
from openai import OpenAI

# Create your PAT token by following instructions here:
# https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
# grab the token from the token.txt file
with open("PAT.txt", "r") as f:
    GITHUB_TOKEN = f.read().strip()

# Set it as an environment variable
os.environ["GITHUB_TOKEN"] = GITHUB_TOKEN


# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

# List of available models
model_choices = ["gpt-4.1", "Meta-Llama-3-70B-Instruct"]
#DeepSeek is not allowed on gov owened machines

In [ ]:
# Zero-shot
# Role-Playing
# Chain-of-Thoughts
# Prompt Chaining
# Self-Consistency
# Few-shot

# CHANGE THIS TO SWITCH MODELS
model = 0; #0 for ChatGPT outputs, 1 for Llama outputs.

1 - Zero Shot

In [ ]:
code = '''
public Map <String , Integer > countWordFrequency(List <String > words) {
 	Map <String , Integer > freqMap = new HashMap <>();
 	for (String word : words) {
     	freqMap.put(word , freqMap.getOrDefault(word , 0) + 1);
 	}
 	return freqMap;
}
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Summarize the functionality of the following method.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

This method takes a list of strings as input and returns a map where the keys are the unique words from the input list and the values are the frequency of each word (i.e., how many times each word appears in the list).

In other words, it counts the frequency of each word in the input list and returns a map with the word frequencies.


1 - Role Playing

In [ ]:
code = '''
public Map <String , Integer > countWordFrequency(List <String > words) {
 	Map <String , Integer > freqMap = new HashMap <>();
 	for (String word : words) {
     	freqMap.put(word , freqMap.getOrDefault(word , 0) + 1);
 	}
 	return freqMap;
}
'''

messages = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": f"I want you to summarize this java method. Here is the code: {code}."}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: This Java method, `countWordFrequency`, takes a list of strings as input and returns a map where the keys are the unique words in the list and the values are their respective frequencies. In other words, it counts the occurrence of each word in the list and returns the result as a map.


2 - Zero Shot

In [ ]:
code = '''
def sum_range(start , end):
 	total = 0
 	for i in range(start , end):
     	total += i
 	return total
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Identify and fix the off-by-one error in this function. Please only return the fixed code.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

def sum_range(start, end):
 	total = 0
 	for i in range(start, end + 1):
     	total += i
 	return total


2 - Prompt Chaining

In [ ]:

code = '''
def sum_range(start , end):
 	total = 0
 	for i in range(start , end):
     	total += i
 	return total
'''

messages = [
    {"role": "user",
     "content": f"This is a function that attempts to calculate the sum of a range of numbers: {code}. Please only analyze it."}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("First chain:\n", reply)

# Add response to the conversation
messages.append({"role": "user", "content": reply})

# User follows up
followup = "The code you analyzed has a an off-by-one error. Where could it be? Please do not try to fix, just find the error."

# Add user message
messages.append({"role": "user", "content": followup})

# Call again with updated history
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

reply2 = response.choices[0].message.content
print("Second chain:\n", reply2)

# Add response to the conversation
messages.append({"role": "user", "content": reply})

# User follows up
followup2 = "Please now provide the fixed code."

# Add user message
messages.append({"role": "user", "content": followup2})

# Call again with updated history
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

print("Third chain:\n", response.choices[0].message.content)

First chain:
 Here's an analysis of the function:

**Correctness:** The function attempts to calculate the sum of a range of numbers from `start` to `end-1` (since the `range` function in Python is exclusive of the end value). However, it does not correctly calculate the sum because it excludes the `end` value itself. To fix this, the function should iterate up to `end+1` or use the `range` function with the `end` value incremented by 1.

**Efficiency:** The function has a time complexity of O(n), where n is the range of numbers being summed. This is because it uses a loop to iterate over each number in the range. While this is acceptable for small ranges, it may become inefficient for large ranges.

**Code quality:** The function has a clear and concise implementation, making it easy to understand. However, it lacks documentation (e.g., a docstring) to explain its purpose and behavior.

**Potential issues:**

* If `start` is greater than `end`, the function will return 0, which may no

3 - Chain of Thought

In [ ]:
code = '''
int* getArray(int size) {
 	int arr[size]; // Warning: local array
 	return arr; // Bug: returning pointer to local variable
}
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Please classify the type of bug in this C++ code. Break it down step by step, please:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Let's break down the bug in this C++ code step by step:

**Step 1: Local array declaration**

`int arr[size];`

Here, an array `arr` is declared with a size determined by the `size` parameter. This is a local array, meaning it is allocated on the stack and has automatic storage duration. This is not a problem in itself, but it sets the stage for the bug.

**Step 2: Returning a pointer to the local array**

`return arr;`

The function returns a pointer to the first element of the local array `arr`. This is where the bug occurs.

**The bug: Returning a pointer to a local variable**

The problem is that the local array `arr` is allocated on the stack, and its lifetime ends when the function returns. When the function returns, the memory occupied by `arr` is released, and the pointer returned by the function becomes invalid.

In C++, when a function returns, its local variables are destroyed, and their memory is reclaimed. This means that the pointer returned by the function points to memo

3 - Self-Consistency

In [ ]:
code = '''
int* getArray(int size) {
 	int arr[size]; // Warning: local array
 	return arr; // Bug: returning pointer to local variable
}
'''

messages = [
    {"role": "user",
     "content": f"Please clasify the bug in this C++ code. However (AND THIS IS IMPORTANT), please generate 3 classifications. These classifications can be the same, but take it as if I gave you the same prompt 3 times in a row. Then, take the aggregate of the classifications you come up with, and give me this.{code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("First chain:\n", reply)

First chain:
 Here are three classifications for the bug in the given C++ code:

**Classification 1:**
* Category: Memory Management
* Type: Dangling Pointer
* Severity: High

**Classification 2:**
* Category: Scope and Lifetime
* Type: Returning Pointer to Local Variable
* Severity: Critical

**Classification 3:**
* Category: Memory Safety
* Type: Use-After-Free
* Severity: High

Aggregate Classification:
* Category: Memory Management/Memory Safety
* Type: Dangling Pointer/Returning Pointer to Local Variable
* Severity: Critical/High

The bug in the code is that it returns a pointer to a local array, which goes out of scope as soon as the function returns. This leads to undefined behavior, as the returned pointer points to memory that is no longer valid. The aggregate classification reflects the consensus among the three individual classifications, highlighting the critical severity of this bug.


4 - Zero Shot

In [ ]:
code = '''
def is_valid_email(email):
 	# TODO: Complete using regex
 	pass
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Complete the function using regex to validate basic email addresses. Please only return the fixed code.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

```
import re

def is_valid_email(email):
    pattern = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
    return bool(re.match(pattern, email))
```


4 - Role-Playing

In [ ]:
code = '''
def is_valid_email(email):
 	# TODO: Complete using regex
 	pass
'''

messages = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": f"I want you to create a Python function that uses regex to validate email addresses. Here is the code: {code}."}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: Here's a Python function that uses regex to validate email addresses:

```python
import re

def is_valid_email(email):
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    if re.match(email_regex, email):
        return True
    else:
        return False
```

This function uses a regular expression pattern to match most common email address formats. Here's a breakdown of the pattern:

- `\b`: Asserts a word boundary.
- `[A-Za-z0-9._%+-]+`: Matches one or more alphanumeric characters, dots, underscores, percent signs, plus signs, or hyphens.
- `@`: Matches the @ symbol.
- `[A-Za-z0-9.-]+`: Matches one or more alphanumeric characters, dots, or hyphens.
- `\.`: Matches a dot.
- `[A-Z|a-z]{2,}`: Matches the domain extension (it must be at least 2 characters long).

You can use this function to validate email addresses like this:

```python
email = "example@example.com"
if is_valid_email(email):
    print("Email is valid")
else:
    print("Email is not va

5 - Zero Shot

In [ ]:
code = '''
from flask import Flask , jsonify

app = Flask(__name__)

@app.route(’/greet/<username >’)
def greet(username):
 	# TODO: Return a JSON greeting
 	pass
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Create a ‘/greet/<username>‘ endpoint that returns a JSON greeting. Please only return the fixed code.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

```python
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    return jsonify({'message': f'Hello, {username}!'})
```


5 - Role-Playing

In [ ]:
code = '''
from flask import Flask , jsonify

app = Flask(__name__)

@app.route(’/greet/<username >’)
def greet(username):
 	# TODO: Return a JSON greeting
 	pass
'''

messages = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": f"Create a ‘/greet/<username>‘ endpoint that returns a JSON greeting.: {code}."}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: Here's the completed code:
```
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    greeting = {'message': f'Hello, {username}!'}
    return jsonify(greeting)

if __name__ == '__main__':
    app.run()
```
Here's what's changed:

* I removed the extra space in the route definition `@app.route('/greet/<username>')`.
* In the `greet` function, I created a dictionary `greeting` with a single key-value pair, where the key is `'message'` and the value is a string that includes the `username` parameter using an f-string.
* I returned the `greeting` dictionary using the `jsonify` function, which will convert the dictionary to a JSON response.

Now, when you send a GET request to `/greet/<username>`, you'll receive a JSON response like this:
```json
{
  "message": "Hello, <username>!"
}
```
Replace `<username>` with the actual username you want to greet!


6 - Zero-Shot

In [ ]:
code = '''
-- TODO: Design schema with appropriate keys and constraints
 -- Tables: users(id, name), books(id, title), reviews(id, user_id , book_id , rating)
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Write the schema for a review app with users, books, and reviews. Use this code to help:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here is a possible schema for a review app with users, books, and reviews:
```
CREATE TABLE users (
  id SERIAL PRIMARY KEY,
  name VARCHAR(50) NOT NULL
);

CREATE TABLE books (
  id SERIAL PRIMARY KEY,
  title VARCHAR(100) NOT NULL
);

CREATE TABLE reviews (
  id SERIAL PRIMARY KEY,
  user_id INTEGER NOT NULL,
  book_id INTEGER NOT NULL,
  rating SMALLINT NOT NULL CHECK (rating >= 1 AND rating <= 5),
  review_text TEXT,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  FOREIGN KEY (user_id) REFERENCES users(id),
  FOREIGN KEY (book_id) REFERENCES books(id)
);
```
Here's a brief explanation of each part:

**users table**

* `id`: a unique identifier for each user, auto-incrementing using `SERIAL`
* `name`: the user's name, with a maximum length of 50 characters

**books table**

* `id`: a unique identifier for each book, auto-incrementing using `SERIAL`
* `title`: the title of the book, with a maximum length of 100 characters

**reviews table**

* `id`: a unique identifier for each r

6 - Role-Playing

In [ ]:
code = '''
-- TODO: Design schema with appropriate keys and constraints
-- Tables: users(id, name), books(id, title), reviews(id, user_id , book_id , rating)
'''

messages = [
    {"role": "system", "content": "You are a helpful SQL coding assistant."},
    {"role": "user", "content": f"Write the schema for a review app with users, books, and reviews. Use this code to help: {code}."}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: Here is the schema for a review app with users, books, and reviews:

```sql
CREATE TABLE users (
  id INTEGER PRIMARY KEY,
  name VARCHAR(50) NOT NULL
);

CREATE TABLE books (
  id INTEGER PRIMARY KEY,
  title VARCHAR(100) NOT NULL
);

CREATE TABLE reviews (
  id INTEGER PRIMARY KEY,
  user_id INTEGER NOT NULL,
  book_id INTEGER NOT NULL,
  rating INTEGER CHECK (rating BETWEEN 1 AND 5),
  FOREIGN KEY (user_id) REFERENCES users(id),
  FOREIGN KEY (book_id) REFERENCES books(id)
);
```

Let me explain what this schema does:

- The `users` table has two columns: `id` and `name`. The `id` column is the primary key, which uniquely identifies each user. The `name` column stores the user's name.

- The `books` table also has two columns: `id` and `title`. The `id` column is the primary key, which uniquely identifies each book. The `title` column stores the book's title.

- The `reviews` table has four columns: `id`, `user_id`, `book_id`, and `rating`. The `id` column is the primary 

7 - Zero-Shot

In [ ]:
code = '''
public int getLength(String s) {
 	return s.length (); // What if s is null?
}
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Identify any null dereference risk in this code:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

There is a null dereference risk in this code. If the input `s` is `null`, calling `s.length()` will throw a `NullPointerException` because you're trying to access the `length` method on a null object.

To mitigate this risk, you can add a null check before calling `length()`:
```
public int getLength(String s) {
    return s != null ? s.length() : 0; // or some other default value
}
```
Alternatively, you can throw a `NullPointerException` explicitly if `s` is null, depending on your requirements:
```
public int getLength(String s) {
    if (s == null) {
        throw new NullPointerException("s cannot be null");
    }
    return s.length();
}
```
Either way, it's essential to handle the possibility of `s` being null to avoid a `NullPointerException` at runtime.


7 - Chain of Thought

In [ ]:
code = '''
public int getLength(String s) {
 	return s.length (); // What if s is null?
}
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Please identify any null deference risk in this code. Break it down step by step.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Let's break down the code step by step to identify the null deference risk:

1. `public int getLength(String s) {`:
	* The method takes a `String` parameter `s`.
	* The method returns an `int` value.
2. `return s.length ();`:
	* The method tries to call the `length()` method on the `s` object.
	* This is where the null deference risk comes in:
		+ If `s` is `null`, calling `length()` on it will throw a `NullPointerException` (NPE).

The risk is that if the caller of this method passes `null` as an argument, the method will attempt to call `length()` on a null reference, resulting in an NPE.

To mitigate this risk, you can add a null check before calling `length()`:
```
public int getLength(String s) {
    if (s == null) {
        return 0; // or throw an exception, depending on your requirements
    }
    return s.length();
}
```
By doing so, you ensure that the method will not throw an NPE if `s` is `null`.


8 - Few-shot

In [ ]:

example1 = '''
import csv
from io import StringIO

def parse_csv_line(line):
    reader = csv.reader(StringIO(line))
    return next(reader)
'''

example2 = '''
def parse_csv_line(line):
    fields = []
    field = ''
    in_quotes = False
    i = 0
    while i < len(line):
        char = line[i]
        if char == '"':
            if in_quotes and i + 1 < len(line) and line[i + 1] == '"':
                # Escaped quote
                field += '"'
                i += 1
            else:
                in_quotes = not in_quotes
        elif char == ',' and not in_quotes:
            fields.append(field)
            field = ''
        else:
            field += char
        i += 1
    fields.append(field)
    return fields
'''
code = '''
def parse_csv_line(line):
 	return line.split(’,’) # Incomplete: doesn’t handle quoted fields
'''

messages = [
    {"role": "user",
     "content": f"Here are two functions that parse a csv line. Please understand them and do not respond with anything yet. Example 1: {example1}. Example 2: {example2}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("First chain:\n", reply)

# Add response to the conversation
messages.append({"role": "user", "content": reply})

# User follows up
followup2 = f"Now, please fill in this this parser, improving it to be able to support quoted fields: {code}"

# Add user message
messages.append({"role": "user", "content": followup2})

# Call again with updated history
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

print("\nSecond chain:\n", response.choices[0].message.content)

First chain:
 I've taken the time to understand the two functions that parse a CSV line. I'll wait for your next instruction before responding.

Second chain:
 Here's the improved parser that supports quoted fields:

```
def parse_csv_line(line):
    fields = []
    field = ''
    in_quotes = False

    for char in line:
        if char == '"':
            if in_quotes and line[line.index(char) + 1] == '"':
                # Escaped quote
                field += '"'
            else:
                in_quotes = not in_quotes
        elif char == ',' and not in_quotes:
            fields.append(field)
            field = ''
        else:
            field += char

    fields.append(field)
    return fields
```

This parser goes through each character in the line, keeping track of whether it's currently inside a quoted field or not. If it encounters a comma outside of a quoted field, it adds the current field to the list of fields and resets the field. If it encounters a quote, it toggl

8 - Chain-of-Thoughts

In [ ]:
code = '''
def parse_csv_line(line):
 	return line.split(’,’) # Incomplete: doesn’t handle quoted fields
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Please write a parser that supports quoted fields. Break it down step by step, please:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here is a step-by-step implementation of a CSV parser that supports quoted fields:

**Step 1: Initialize an empty list to store the parsed fields**
```
def parse_csv_line(line):
    fields = []
```
**Step 2: Initialize a flag to track whether we're inside a quoted field**
```
    in_quote = False
```
**Step 3: Iterate through each character in the input line**
```
    for char in line:
```
**Step 4: Handle the case where we're not inside a quoted field**
```
        if not in_quote:
            if char == ',':
                # We've reached a comma, so add the current field to the list
                fields.append('')
            elif char == '"':
                # We've reached a quote, so set the flag and move on
                in_quote = True
            else:
                # We've reached a regular character, so add it to the current field
                fields[-1] += char
```
**Step 5: Handle the case where we're inside a quoted field**
```
        else:
            if char 

9 - Zero Shot

In [ ]:
code = '''
data class Product(val id: Int , val name: String , val price: Double)

// TODO: Create GET and POST endpoints using Ktor
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Convert the data class to a REST API using Ktor:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here's an example of how you can convert the `Product` data class to a REST API using Ktor:

First, add the Ktor dependencies to your `build.gradle` file (if you're using Gradle):
```groovy
dependencies {
    implementation 'io.ktor:ktor-server-netty:1.6.7'
    implementation 'io.ktor:ktor-webjars:1.6.7'
    implementation 'io.ktor:ktor-gson:1.6.7'
}
```
Or, if you're using Maven, add the following dependencies to your `pom.xml` file:
```xml
<dependencies>
    <dependency>
        <groupId>io.ktor</groupId>
        <artifactId>ktor-server-netty</artifactId>
        <version>1.6.7</version>
    </dependency>
    <dependency>
        <groupId>io.ktor</groupId>
        <artifactId>ktor-webjars</artifactId>
        <version>1.6.7</version>
    </dependency>
    <dependency>
        <groupId>io.ktor</groupId>
        <artifactId>ktor-gson</artifactId>
        <version>1.6.7</version>
    </dependency>
</dependencies>
```
Next, create a Ktor application class:
```kotlin
import io.ktor.applic

9 - Self Consistency

In [ ]:
code = '''
data class Product(val id: Int , val name: String , val price: Double)

// TODO: Create GET and POST endpoints using Ktor
'''

messages = [
    {"role": "user",
     "content": f"Please convert the data class to a REST API using Ktor. However (AND THIS IS IMPORTANT), please generate 3 classifications. These classifications can be the same, but take it as if I gave you the same prompt 3 times in a row. Then, take the aggregate of the classifications you come up with, and give me this.{code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("First chain:\n", reply)

First chain:
 Certainly! Here’s how I’ll proceed:

1. **Classifications** — I interpret “classification” as possible ways to structure or organize the REST API endpoints for the Product data class (e.g., classic CRUD, minimal REST, with or without validation, etc).  
2. **Three Classifications** — I’ll generate three REST API implementations, each potentially differing in detail (but all using Ktor, GET and POST, with your Product data class).
3. **Aggregate** — I’ll then combine/aggregate the best or most representative features from the three into a single, recommended implementation.

---

### **Classification 1: Simple In-memory List**

- Stores products in a mutable list.
- GET returns all products.
- POST adds a new product.

```kotlin
import io.ktor.application.*
import io.ktor.response.*
import io.ktor.request.*
import io.ktor.routing.*
import io.ktor.features.*
import io.ktor.serialization.*

data class Product(val id: Int, val name: String, val price: Double)

fun Application

10 - Chain-of-Thoughts

In [ ]:
code = '''
def reverse_words(sentence):
 	return ’ ’.join(sentence.split()[:: -1])
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Please write a brief summary of this function. Break it down step by step.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here is a brief summary of the `reverse_words` function, broken down step by step:

**Summary:** The `reverse_words` function takes a sentence as input and returns the sentence with the words in reverse order.

**Step-by-Step Breakdown:**

1. `sentence.split()`: This splits the input sentence into a list of individual words, using whitespace as the separator. For example, if the input sentence is "Hello World", this would result in a list `["Hello", "World"]`.
2. `[:: -1]`: This is a slice notation that reverses the order of the list. The `::` means "start from the beginning and go to the end", and the `-1` means "step backwards by 1 element". So, this reverses the list of words. In our example, the list would become `["World", "Hello"]`.
3. `' '.join(...)`: This takes the reversed list of words and joins them back together into a single string, using a space (`' '`) as the separator. In our example, the resulting string would be "World Hello".

So, the final output of the `reverse_wor

10 - Role-Playing

In [ ]:
code = '''
def reverse_words(sentence):
 	return ’ ’.join(sentence.split()[:: -1])
'''


messages = [
    {"role": "system", "content": "You are a helpful coding assistant, one that is great at summarizing code."},
    {"role": "user", "content": f"Please write a brief summary of this function:\n{code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: Here is a brief summary of the function:

This function, `reverse_words`, takes a sentence as input and returns the sentence with the order of its words reversed. For example, "hello world" would become "world hello".


11 - Zero Shot

In [ ]:
code = '''
# This function checks if a number is prime
 def is_prime(n):
 	if n <= 1:
     	return False
 	for i in range(2, int(n**0.5) +1):
     	if n % i == 0:
         	return False
 	return True
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Generate a prompt that you could feed to a large language model to produce this code:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here is a prompt that could generate the given code:

"Write a Python function named `is_prime` that takes an integer `n` as input and returns `True` if `n` is a prime number, and `False` otherwise. The function should use a efficient algorithm to check for primality."

Alternatively, you could also use a more concise prompt:

"Python function to check if a number is prime"

Or, if you want to provide more guidance on the algorithm to use:

"Write a Python function to check if a number is prime using a trial division method with an optimization to only check up to the square root of the number."


11 - Chain of Thought

In [ ]:
code = '''
# This function checks if a number is prime
 def is_prime(n):
 	if n <= 1:
     	return False
 	for i in range(2, int(n**0.5) +1):
     	if n % i == 0:
         	return False
 	return True
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Generate a prompt that you could feed to a large language model to produce this code. Break it down step by step.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here's a prompt that could be fed to a large language model to produce the given code:

**Prompt:** "Write a Python function named `is_prime` that takes an integer `n` as input and returns `True` if `n` is a prime number, and `False` otherwise. Implement the function using a simple trial division method."

Let's break it down step by step:

1. **Function signature**: "Write a Python function named `is_prime`" - This tells the model to generate a Python function with the specified name.
2. **Function parameters**: "that takes an integer `n` as input" - This specifies the input parameter type and name.
3. **Function purpose**: "and returns `True` if `n` is a prime number, and `False` otherwise" - This describes the function's behavior and output.
4. **Implementation hint**: "Implement the function using a simple trial division method" - This gives the model a hint about the algorithm to use, which is a common approach for checking primality.

By providing this prompt, the language model 

12 - Zero-Shot

In [ ]:
code = '''
def factorial(n):
 	result = 1
 	for i in range(1, n):
     	result *= i
 	return result
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Fix the bug when n is 0:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

The bug in the code is that when `n` is 0, the function will return 1, but the factorial of 0 is defined to be 1. Also, the loop will not execute when `n` is 0, so the function will return 1, which is correct. However, the loop should iterate from 1 to `n` (inclusive), so the range should be `range(1, n+1)`.

Here is the corrected code:

```
def factorial(n):
    if n < 0:
        return "Error: Factorial is not defined for negative numbers"
    elif n == 0:
        return 1
    else:
        result = 1
        for i in range(1, n+1):
            result *= i
        return result
```

This code will correctly return the factorial of `n` for non-negative integers `n`. It will also return an error message if `n` is negative, since the factorial is not defined for negative numbers.


12 - Prompt Chaining

In [ ]:

code = '''
def factorial(n):
 	result = 1
 	for i in range(1, n):
     	result *= i
 	return result
'''

messages = [
    {"role": "user",
     "content": f"This is a function that attempts to find the factorial of a number: {code}. Please only analyze it."}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("First chain:\n", reply)

# Add response to the conversation
messages.append({"role": "user", "content": reply})

# User follows up
followup = "The code you analyzed has an error. Where could it be? Please do not try to fix, just find the error."

# Add user message
messages.append({"role": "user", "content": followup})

# Call again with updated history
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

reply2 = response.choices[0].message.content
print("\nSecond chain:\n", reply2)

# Add response to the conversation
messages.append({"role": "user", "content": reply})

# User follows up
followup2 = "Please now provide the fixed code, knowing that the error arises when n is equal to 0."

# Add user message
messages.append({"role": "user", "content": followup2})

# Call again with updated history
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

print("\nThird chain:\n", response.choices[0].message.content)

First chain:
 Here's an analysis of the function:

**Correctness:** The function is almost correct, but it has a small mistake. The loop should iterate up to `n+1`, not `n`. This is because the `range` function in Python is exclusive of the end value, so `range(1, n)` will only go up to `n-1`. As a result, the function will return an incorrect result for `n > 1`.

**Efficiency:** The function has a time complexity of O(n), which is acceptable for small values of `n`. However, for large values of `n`, the function will become slow and may even cause an overflow error.

**Code style:** The code is well-formatted and easy to read. The variable names are descriptive, and the function has a clear and concise purpose.

**Edge cases:** The function does not handle edge cases such as:

* `n` being a non-integer value (e.g. `n = 3.5`)
* `n` being a negative integer (e.g. `n = -3`)
* `n` being 0 (in which case the factorial is defined to be 1)

Overall, the function is close to being correct, bu

13 - Zero Shot

In [ ]:
code = '''
struct Node {
 	int data;
 	struct Node* next;
};

void deleteNode(struct Node** head , int key) {
 	// TODO: Implement node deletion
}
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Implement node deletion by value:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Certainly! Here's how you can implement deletion of the first occurrence of a node with a given value (`key`) from a singly linked list in C.

```c
#include <stdio.h>
#include <stdlib.h>

struct Node {
    int data;
    struct Node* next;
};

void deleteNode(struct Node** head, int key) {
    struct Node* temp = *head;
    struct Node* prev = NULL;

    // If head node holds the key
    if (temp != NULL && temp->data == key) {
        *head = temp->next; // Move head
        free(temp);         // Free old head
        return;
    }

    // Search for the key, keep track of previous node
    while (temp != NULL && temp->data != key) {
        prev = temp;
        temp = temp->next;
    }

    // If key wasn't present in linked list
    if (temp == NULL)
        return;

    // Unlink the node from linked list
    prev->next = temp->next;
    free(temp);
}
```

**Explanation:**

1. **Check if the head node contains the key:**  
   If so, move the head pointer to the next node and free t

13 - Few Shot

In [ ]:
example1 = '''
void deleteAllByValue(Node** head, int value) {
    Node *temp = *head, *prev = NULL;

    // Delete nodes from the beginning if they match
    while (temp != NULL && temp->data == value) {
        *head = temp->next;
        free(temp);
        temp = *head;
    }

    // Now handle other nodes
    while (temp != NULL) {
        while (temp != NULL && temp->data != value) {
            prev = temp;
            temp = temp->next;
        }

        if (temp == NULL) return;

        prev->next = temp->next;
        free(temp);
        temp = prev->next;
    }
}
'''

example2 = '''
// Delete first occurrence of a given value
void deleteByValue(Node** head, int value) {
    Node *temp = *head, *prev = NULL;

    // If head node holds the value
    if (temp != NULL && temp->data == value) {
        *head = temp->next;
        free(temp);
        return;
    }

    // Search for the value
    while (temp != NULL && temp->data != value) {
        prev = temp;
        temp = temp->next;
    }

    // Value not found
    if (temp == NULL) return;

    // Unlink and delete
    prev->next = temp->next;
    free(temp);
}
'''
code = '''
struct Node {
 	int data;
 	struct Node* next;
};

void deleteNode(struct Node** head , int key) {
 	// TODO: Implement node deletion
}
'''

messages = [
    {"role": "user",
     "content": f"Here are two functions that delete linked list nodes by value. Please understand them and do not respond with anything yet. Example 1: {example1}. Example 2: {example2}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("First chain:\n", reply)

# Add response to the conversation
messages.append({"role": "user", "content": reply})

# User follows up
followup2 = f"Now, please complete this method to delete nodes by value: {code}"

# Add user message
messages.append({"role": "user", "content": followup2})

# Call again with updated history
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

print("\nSecond chain:\n", response.choices[0].message.content)

First chain:
 Understood! I have reviewed both functions:

- `deleteAllByValue` deletes **all** nodes in a linked list that match a given value.
- `deleteByValue` deletes **only the first occurrence** of a node with the given value.

Both functions handle the head node as a special case and use a combination of `prev` and `temp` pointers to traverse and delete nodes. The functions use `free()` to deallocate memory of the deleted nodes.

I'm ready to answer any questions or perform any task related to these functions!

Second chain:
 Here is the implementation of the `deleteNode` method to delete nodes by value:

```c
void deleteNode(struct Node** head, int key) {
    struct Node *temp = *head, *prev = NULL;

    // If head node holds the key, change head
    if (temp != NULL && temp->data == key) {
        *head = temp->next;
        free(temp);
        return;
    }

    // Search for the key
    while (temp != NULL && temp->data != key) {
        prev = temp;
        temp = temp->nex

14 - Zero-Shot

In [ ]:
code = '''
def fibonacci(n):
 	# TODO: Base cases and recursive call
 	pass
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Complete the recursive function for Fibonacci:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here is the completed recursive function for Fibonacci:
```
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```
Here's a brief explanation:

* The base cases are `n == 0` and `n == 1`, which return 0 and 1, respectively. These are the initial values of the Fibonacci sequence.
* For `n > 1`, we make a recursive call to `fibonacci(n-1)` and `fibonacci(n-2)`, and add their results together. This is because the `n`-th Fibonacci number is the sum of the `(n-1)`-th and `(n-2)`-th Fibonacci numbers.

Note that this implementation has an exponential time complexity due to the repeated recursive calls, which can lead to performance issues for large values of `n`. A more efficient approach would be to use an iterative solution or memoization to store previously computed values.


14 - Role-Playing

In [ ]:
code = '''
def fibonacci(n):
 	# TODO: Base cases and recursive call
 	pass
'''


messages = [
    {"role": "system", "content": "You are a helpful coding assistant, one that is great at providing correct code snippets."},
    {"role": "user", "content": f"Please complete the recursive function for Fibonacci:\n{code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: Here is the complete recursive function for Fibonacci:
```
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```
This function uses the standard recursive formula for Fibonacci numbers: `fib(n) = fib(n-1) + fib(n-2)`. The base cases are `fib(0) = 0` and `fib(1) = 1`.

Note that this function has an exponential time complexity, which can make it slow for large values of `n`. If you need a more efficient solution, you may want to consider using an iterative approach or memoization.


15 - Zero Shot

In [ ]:
code = '''
class Person:
 	def __init__(self):
     	# TODO: Add name , age , and optional email
     	pass
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Complete the class constructor:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here is the completed class constructor:

```
class Person:
    def __init__(self, name, age, email=None):
        self.name = name
        self.age = age
        self.email = email
```

In this constructor, `name` and `age` are required parameters, and `email` is an optional parameter with a default value of `None`. This means that when creating a new `Person` object, you must provide a `name` and an `age`, but you can optionally provide an `email`.


15 - Self Consistency

In [ ]:
code = '''
class Person:
 	def __init__(self):
     	# TODO: Add name , age , and optional email
     	pass
'''

messages = [
    {"role": "user",
     "content": f"Please complete this class constructor. However (AND THIS IS IMPORTANT), please generate 3 classifications. These classifications can be the same, but take it as if I gave you the same prompt 3 times in a row. Then, take the aggregate of the classifications you come up with, and give me this.{code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("First chain:\n", reply)

First chain:
 Here are three different classifications for the `Person` class constructor:

**Classification 1**
```
class Person:
    def __init__(self, name: str, age: int, email: str = None):
        self.name = name
        self.age = age
        self.email = email
```
**Classification 2**
```
class Person:
    def __init__(self, name: str, age: int, email: str = "not_provided"):
        self.name = name
        self.age = age
        self.email = email
```
**Classification 3**
```
class Person:
    def __init__(self, name: str, age: int, email: str = None):
        self.full_name = name
        self.age_in_years = age
        self.contact_email = email
```
Now, let's take the aggregate of these classifications:

* All three classifications agree on the `name` and `age` parameters, with `name` being a string and `age` being an integer.
* Two out of three classifications use `email` as an optional parameter with a default value of `None`. The third classification uses `email` with a

16 - Few-Shot

In [ ]:

example1 = '''
    int binarySearch(int[] arr, int target) {
        int low = 0;
        int high = arr.length - 1;

        while (low <= high) {
            int mid = low + (high - low) / 2; // to prevent potential overflow

            if (arr[mid] == target) {
                return mid; // Target found
            } else if (arr[mid] < target) {
                low = mid + 1; // Search in the right half
            } else {
                high = mid - 1; // Search in the left half
            }
        }
        return -1; // Target not found
    }
'''

example2 = '''
int binarySearch(int arr[], int x)
    {
        int low = 0, high = arr.length - 1;
        while (low <= high) {
            int mid = low + (high - low) / 2;

            // Check if x is present at mid
            if (arr[mid] == x)
                return mid;

            // If x greater, ignore left half
            if (arr[mid] < x)
                low = mid + 1;

            // If x is smaller, ignore right half
            else
                high = mid - 1;
        }

        // If we reach here, then element was
        // not present
        return -1;
    }
'''
code = '''
public int binarySearch(int[] arr , int target) {
 	int left = 0, right = arr.length - 1;
 	while (left <= right) {
     	int mid = (left + right) / 2;
     	// TODO: Compare and adjust bounds
 	}
 	return -1;
 }
'''

messages = [
    {"role": "user",
     "content": f"Here are two functions that Implement a binary search in Java. Please understand them. Example 1: {example1}. Example 2: {example2} \n Now, finish this code: {code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print(reply)

Here is the completed code:

```java
public int binarySearch(int[] arr, int target) {
    int left = 0, right = arr.length - 1;
    while (left <= right) {
        int mid = left + (right - left) / 2; // to prevent potential overflow

        if (arr[mid] == target) {
            return mid; // Target found
        } else if (arr[mid] < target) {
            left = mid + 1; // Search in the right half
        } else {
            right = mid - 1; // Search in the left half
        }
    }
    return -1; // Target not found
}
```

This code implements a binary search algorithm to find the index of a target element in a sorted array. Here's a step-by-step explanation:

1. Initialize two pointers, `left` and `right`, to the start and end of the array, respectively.
2. Loop until `left` is greater than `right`.
3. Calculate the midpoint `mid` of the current range `[left, right]`.
4. Compare the element at `mid` with the `target`:
	* If they are equal, return `mid` as the index of the targe

16 - Self-Consistency

In [ ]:
code = '''
public int binarySearch(int[] arr , int target) {
 	int left = 0, right = arr.length - 1;
 	while (left <= right) {
     	int mid = (left + right) / 2;
     	// TODO: Compare and adjust bounds
 	}
 	return -1;
 }
'''

messages = [
    {"role": "user",
     "content": f"Please implement the binary search algorithm in Java. However (AND THIS IS IMPORTANT), please generate 3 implementations. These implementations can be the same, but take it as if I gave you the same prompt 3 times in a row. Then, take the aggregate of the implementations you come up with, and give me this.{code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("First chain:\n", reply)

First chain:
 Here are three implementations of the binary search algorithm in Java:

**Implementation 1**
```
public int binarySearch(int[] arr, int target) {
    int left = 0, right = arr.length - 1;
    while (left <= right) {
        int mid = (left + right) / 2;
        if (arr[mid] == target) {
            return mid;
        } else if (arr[mid] < target) {
            left = mid + 1;
        } else {
            right = mid - 1;
        }
    }
    return -1;
}
```
**Implementation 2**
```
public int binarySearch(int[] arr, int target) {
    int left = 0, right = arr.length - 1;
    while (left <= right) {
        int mid = (left + right) / 2;
        if (arr[mid] == target) {
            return mid;
        } else if (target < arr[mid]) {
            right = mid - 1;
        } else {
            left = mid + 1;
        }
    }
    return -1;
}
```
**Implementation 3**
```
public int binarySearch(int[] arr, int target) {
    int left = 0, right = arr.length - 1;
    while (left 

17 - Zero Shot

In [ ]:
code = '''
// Supposed to return true if x is even
bool isOdd(int x) {
 	return x % 2 == 0; // Logic contradicts function name
}
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Please resolve the inconsistency between function name and logic:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

The function name `isOdd` suggests that it should return `true` if the input `x` is odd, but the logic `x % 2 == 0` returns `true` if `x` is even. To resolve this inconsistency, you can simply change the function name to `isEven` or flip the logic to `x % 2 != 0`. Here are both options:

**Option 1: Change function name to `isEven`**
```c
bool isEven(int x) {
  return x % 2 == 0; // Correct logic for even numbers
}
```
**Option 2: Flip the logic to `x % 2 != 0`**
```c
bool isOdd(int x) {
  return x % 2 != 0; // Correct logic for odd numbers
}
```
Either of these changes will resolve the inconsistency between the function name and the logic.


17 - Role Playing

In [ ]:
code = '''
// Supposed to return true if x is even
bool isOdd(int x) {
 	return x % 2 == 0; // Logic contradicts function name
}
'''

messages = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": f"I want you to fix the inconsistency in this method. Here is the code: {code}."}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: The method `isOdd` is supposed to return `true` if the number is odd, but the logic inside the method is checking if the number is even. Here's the corrected code:

```c++
bool isOdd(int x) {
  return x % 2 != 0; // Return true if x is odd
}
```

Alternatively, if you want to keep the original logic but make the method name consistent, you could rename the method to `isEven`:

```c++
bool isEven(int x) {
  return x % 2 == 0; // Return true if x is even
}
```


18 - Prompt-Chaining

In [ ]:

code = '''
function isEven(n) {
 	return n % 2; // Returns 1 or 0, not true/false
 }
'''

messages = [
    {"role": "user",
     "content": f"This is a JavaScript function that attempts to find out if a number is even or not: {code}. Please only analyze it."}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

reply = response.choices[0].message.content
print("First chain:\n", reply)

messages.append({"role": "user", "content": reply})

followup = "The code you analyzed has an error. Where could it be? Please do not try to fix, just find the error."

messages.append({"role": "user", "content": followup})

response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

reply2 = response.choices[0].message.content
print("\nSecond chain:\n", reply2)

messages.append({"role": "user", "content": reply})

followup2 = "Please now provide the fixed code."

messages.append({"role": "user", "content": followup2})

response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

print("\nThird chain:\n", response.choices[0].message.content)

First chain:
 Let's break down the `isEven` function:

* The function takes a single argument `n`, which is expected to be a number.
* The function returns the result of the expression `n % 2`.
* The `%` operator is the modulo operator, which returns the remainder of dividing `n` by 2.
* When `n` is even, the remainder of dividing it by 2 is 0.
* When `n` is odd, the remainder of dividing it by 2 is 1.

The issue with this implementation is that it returns an integer value (0 or 1) instead of a boolean value (true or false) indicating whether the number is even or not. This means that if you were to use this function in a conditional statement, it would not behave as expected.

For example:
```
if (isEven(4)) {
  console.log("4 is even"); // This would not log anything, because 0 is falsy
}

if (isEven(3)) {
  console.log("3 is even"); // This would log, because 1 is truthy
}
```
To fix this, the function should return a boolean value, such as:
```
function isEven(n) {
  return n % 2 =

18 - Zero-Shot

In [ ]:
code = '''
function isEven(n) {
 	return n % 2; // Returns 1 or 0, not true/false
 }
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Identify and fix the bug in this JavaScript code:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

The bug in this code is that the function `isEven` is supposed to return a boolean value (true or false) indicating whether the input number `n` is even or not, but instead, it returns the result of the modulo operation `n % 2`, which is an integer value (0 or 1).

To fix the bug, you can modify the function to return a boolean value based on the result of the modulo operation. Here's the corrected code:

```javascript
function isEven(n) {
  return n % 2 === 0; // Returns true if n is even, false otherwise
}
```

In this corrected version, the function returns `true` if `n` is even (i.e., `n % 2` equals 0) and `false` otherwise.


19 - Zero-Shot

In [ ]:
code = '''
// Function that validates an input , calculates square , and returns result
 int process(int x) {
 	if (x < 0) return -1;
 	return x * x;
 }
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Decompose the comment into logical steps.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here are the logical steps broken down from the given comment and code:

**Step 1: Validate the input**

* Check if the input `x` is less than 0.
* If `x` is less than 0, return an error code `-1` immediately.

**Step 2: Calculate the square**

* If `x` is not less than 0, proceed to calculate the square of `x`.
* The square of `x` is calculated as `x * x`.

**Step 3: Return the result**

* Return the calculated square of `x` as the result of the function.

In summary, the function `process` takes an integer `x` as input, checks if it's valid (non-negative), and if so, calculates and returns its square. If the input is invalid, it returns an error code `-1`.


19 - Role Playing

In [ ]:
code = '''
// Function that validates an input , calculates square , and returns result
 int process(int x) {
 	if (x < 0) return -1;
 	return x * x;
 }
'''


messages = [
    {"role": "system", "content": "You are a helpful coding assistant, one that is great at providing correct explanations."},
    {"role": "user", "content": f"Decompose the comment into logical steps.:\n{code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: Here are the logical steps broken down from the given comment and function:

**Step 1: Validate the input**

* Check if the input `x` is less than 0.
* If `x` is less than 0, return -1 (indicating an invalid input).

**Step 2: Calculate the square**

* If `x` is valid (i.e., not less than 0), calculate the square of `x` by multiplying it with itself (`x * x`).

**Step 3: Return the result**

* Return the calculated square of `x` as the result of the `process` function.

In summary, the `process` function takes an integer `x` as input, checks if it's valid (non-negative), and if so, calculates and returns its square.


20 - Role-Playing

In [ ]:
code = '''
def calculate_average(scores):
 	total = 0
 	# TODO: Complete to return average
 	pass
'''


messages = [
    {"role": "system", "content": "You are a helpful coding assistant, one that is great at completing Python functions."},
    {"role": "user", "content": f"Please complete the function based on intent:\n{code}"}
]

# Call the model
response = client.chat.completions.create(
    model=model_choices[model],
    max_tokens=1024,
    temperature=0.7,
    messages=messages
)

# Print the response
reply = response.choices[0].message.content
print("Assistant:", reply)

Assistant: Here is the completed function:

```
def calculate_average(scores):
  total = 0
  for score in scores:
    total += score
  return total / len(scores)
```

This function takes in a list of scores, adds them all up, and then divides by the number of scores to get the average.


20 - Chain-of-Thoughts

In [ ]:
code = '''
def calculate_average(scores):
 	total = 0
 	# TODO: Complete to return average
 	pass
'''

response = client.chat.completions.create(
    model = model_choices[model],
    messages=[
        {"role": "user", "content": f"Please complete the function based on intent. Break it down step by step.:\n{code}"},
    ],
    max_tokens=1024,
    temperature=0.7,
)

print(response.choices[0].message.content)

Here is the completed function:

```
def calculate_average(scores):
  total = 0
  for score in scores:
    total += score
  average = total / len(scores)
  return average
```

Let's break it down step by step:

1. `total = 0`: Initialize a variable `total` to zero. This variable will be used to sum up all the scores.

2. `for score in scores:`: Loop through each score in the `scores` list.

3. `total += score`: In each iteration of the loop, add the current score to the `total`.

4. `average = total / len(scores)`: After the loop, calculate the average by dividing the `total` by the number of scores (which is the length of the `scores` list).

5. `return average`: Return the calculated average.

Note: This function assumes that the `scores` list is not empty. If there's a possibility of an empty list, you should add a check at the beginning of the function to handle this case.
